In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/archive3/archive.zip'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import os

# Chemin du dossier que vous souhaitez explorer
dossier = '/content/drive/MyDrive/ASLdata/asl_alphabet_train/asl_alphabet_train/Z'

# Listez tous les fichiers et dossiers dans le dossier spécifié
contenu_dossier = os.listdir(dossier)

# Affichez toutes les informations sur chaque élément dans le dossier
for element in contenu_dossier:
    chemin_complet = os.path.join(dossier, element)
    stat = os.stat(chemin_complet)
    print(f"Nom: {element}")
    print(f"Chemin complet: {chemin_complet}")
    print(f"Taille: {stat.st_size} octets")
    print(f"Dernière modification: {stat.st_mtime}")
    print("--------------------------")


In [ ]:
A M S Y

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/archive3/archive.zip' -d '/content/drive/MyDrive/ASL'

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing19.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing190.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1900.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1901.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1902.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1903.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1904.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing1905.jpg  
  inflating: /content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train/nothing/nothing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

from sklearn.metrics import accuracy_score, f1_score

In [2]:
def imgPaths(filepath):

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)

    return df

In [3]:
train_image_dir = Path('/content/drive/MyDrive/ASL/asl_alphabet_train/asl_alphabet_train')
train_filepaths = list(train_image_dir.glob(r'**/*.jpg'))

test_image_dir = Path('/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test')
test_filepaths = list(train_image_dir.glob(r'**/*.jpg'))

# Créer un dataframe
train_df = imgPaths(train_filepaths)
test_df = imgPaths(test_filepaths)

# Réduire à 500 images par classe
train_df = train_df.groupby('Label').head(250).reset_index(drop=True)

# Créer un dataframe avec des étiquettes uniques
df_unique = train_df.copy().drop_duplicates(subset=["Label"]).reset_index()

In [4]:
# Définir une fonction pour extraire les étiquettes des chemins de fichiers
def extract_label(filepath):
    return filepath.parts[-2]  # Le nom du dossier parent est considéré comme l'étiquette

# Créer une liste des étiquettes uniques à partir des chemins d'images d'entraînement
unique_labels = list(set(map(extract_label, train_filepaths)))

# Écrire les étiquettes dans un fichier label.txt
with open('label.txt', 'w') as f:
    for label in unique_labels:
        f.write(label + '\n')

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    brightness_range=(0.75, 1.3),
    rotation_range=20,
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    brightness_range=(0.75, 1.3),
    rotation_range=20,
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False
)

# pretrainedModel = tf.keras.applications.MobileNetV2(
#     input_shape=(224, 224, 3),
#     include_top=False,
#     weights='imagenet',
#     pooling='avg'
# )

# pretrainedModel.trainable = False

# Charger le modèle MobileNetV3 avec un facteur de réduction de 0,75
pretrainedModel = tf.keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)


Found 5800 validated image filenames belonging to 29 classes.
Found 1450 validated image filenames belonging to 29 classes.
Found 86126 validated image filenames belonging to 29 classes.
4334752/4334752 [==============================] - 1s 0us/step


In [7]:
pretrainedModel.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 112, 112, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 112, 112, 16)         64        ['Conv[0][0]']                
 lization)                                                                         

In [8]:
inputs = pretrainedModel.input

#x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
#x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(29, activation='softmax')(pretrainedModel.output)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=50,
    decay_rate=0.9,
    staircase=True)

adam = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=12,
    # callbacks=[
    #      tf.keras.callbacks.EarlyStopping(
    #         monitor='val_loss',
    #         patience=2,
    #         restore_best_weights=True
    #     )
    # ]
 )

Epoch 1/12
91/91 [==============================] - 3559s 39s/step - loss: 0.9671 - accuracy: 0.7388 - val_loss: 0.6588 - val_accuracy: 0.7959
Epoch 2/12
91/91 [==============================] - 432s 5s/step - loss: 0.1008 - accuracy: 0.9714 - val_loss: 0.6030 - val_accuracy: 0.8214
Epoch 3/12
91/91 [==============================] - 430s 5s/step - loss: 0.0519 - accuracy: 0.9836 - val_loss: 0.4197 - val_accuracy: 0.8724
Epoch 4/12
91/91 [==============================] - 427s 5s/step - loss: 0.0225 - accuracy: 0.9955 - val_loss: 0.2683 - val_accuracy: 0.9241
Epoch 5/12
91/91 [==============================] - 388s 4s/step - loss: 0.0153 - accuracy: 0.9967 - val_loss: 0.1927 - val_accuracy: 0.9503
Epoch 6/12
91/91 [==============================] - 387s 4s/step - loss: 0.0101 - accuracy: 0.9974 - val_loss: 0.1628 - val_accuracy: 0.9483
Epoch 7/12
91/91 [==============================] - 386s 4s/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.1139 - val_accuracy: 0.9683
Epoch 8/12


In [9]:
# results = model.evaluate(test_images, verbose=0)

model.save('SignLanguageALSClassifier16.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
tf.saved_model.save(model, "/content/drive/MyDrive/model6")

In [36]:
import tensorflow as tf

# Charger le modèle enregistré
modele_charge = tf.saved_model.load("/content/drive/MyDrive/model6")

# Convertir le modèle en format TFLite avec quantification et normalisation
convertisseur = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/model6")
convertisseur.optimizations = [tf.lite.Optimize.DEFAULT]
convertisseur.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Définir la forme des données d'entrée
forme_donnees_entree = (1, 224, 224, 3)  # Format : (batch_size, height, width, channels)

# Définir l'ensemble de données représentatif pour la quantification
def ensemble_donnees_representatif():
    for _ in range(100):  # Choisissez le nombre d'échantillons représentatifs souhaité
        yield [tf.constant(0.0, shape=forme_donnees_entree, dtype=tf.float32)]

convertisseur.representative_dataset = tf.lite.RepresentativeDataset(ensemble_donnees_representatif)
modele_tflite = convertisseur.convert()

# Enregistrer le modèle converti
with open('modele_converti.tflite', 'wb') as f:
    f.write(modele_tflite)


In [66]:
import tensorflow as tf
from PIL import Image
import numpy as np

# Charger le modèle TFLite converti
modele_tflite = "/content/modele_converti.tflite"
interpreteur = tf.lite.Interpreter(model_path=modele_tflite)
interpreteur.allocate_tensors()

# Obtenir les détails des tenseurs d'entrée et de sortie
details_entree = interpreteur.get_input_details()
details_sortie = interpreteur.get_output_details()

# Charger l'image test
chemin_image_test = "/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test/A_test.jpg"
image_test1 = Image.open(chemin_image_test)
image_test1 = image_test1.resize((224, 224))  # Redimensionner l'image à la taille attendue par le modèle
image_test1 = tf.keras.preprocessing.image.img_to_array(image_test1)
image_test1 = tf.expand_dims(image_test1, axis=0)  # Ajouter la dimension du lot

# Normaliser les valeurs de pixel dans la plage [0, 1]
image_test1 = image_test1 / 255.0

# Envoyer l'image test au modèle TFLite
interpreteur.set_tensor(details_entree[0]['index'], image_test1)

# Effectuer l'inférence
interpreteur.invoke()

# Récupérer la sortie
sortie = interpreteur.get_tensor(details_sortie[0]['index'])

# Trouver la classe prédite
classe_predite = np.argmax(sortie)

# Afficher la classe prédite
print("Classe prédite :", classe_predite)


Classe prédite : 4


In [55]:
# Convertir le modèle TensorFlow en TFLite avec quantification et normalisation
convertisseur = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/model6")
convertisseur.optimizations = [tf.lite.Optimize.DEFAULT]  # Appliquer la quantification et la normalisation
convertisseur.target_spec.supported_types = [tf.float32] # optional
tflite_modele = convertisseur.convert()

# Enregistrer le modèle TFLite
chemin_tflite_modele = "/content/modele66.tflite"
with open(chemin_tflite_modele, 'wb') as f:
    f.write(tflite_modele)

In [68]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Charger le modèle TFLite
interpreter = tf.lite.Interpreter(model_path="/content/modele66.tflite")
interpreter.allocate_tensors()

# Obtenir les détails des données d'entrée et de sortie
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Charger une image pour la prédiction
image2 = Image.open("/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test/B_test.jpg")  # Remplacer "image.jpg" par le chemin de votre image

# Prétraiter l'image pour l'adapter aux exigences du modèle
image2 = image2.resize((224, 224))  # Redimensionner l'image selon les besoins du modèle
image_array2 = np.array(image2)  # Convertir l'image en tableau numpy
image_array2 = (image_array2.astype(np.float32) / 127.5) - 1  # Normaliser l'image

# Vérifier le type de données d'entrée attendu par le modèle
if input_details[0]['dtype'] == np.float32:
    image_array2 = np.float32(image)

# Entrer les données d'entrée dans le modèle TFLite
interpreter.set_tensor(input_details[0]['index'], np.expand_dims(image_array2, axis=0))

# Effectuer l'inférence
interpreter.invoke()

# Obtenir les résultats de l'inférence
output_data = interpreter.get_tensor(output_details[0]['index'])

# Sélectionner l'index de la classe avec la probabilité la plus élevée
index_classe_max = np.argmax(output_data)

# Obtenir la probabilité de la classe prédite en pourcentage
probabilite_max = np.max(output_data) * 100

# Afficher le résultat
print("Classe prédite:", index_classe_max)
print("Probabilité de la classe prédite en pourcentage:", output)


Classe prédite: 0
Probabilité de la classe prédite en pourcentage: [[0.03024656 0.00826467 0.00588355 0.04896422 0.02545064 0.01389448
  0.01670773 0.04814664 0.01265362 0.02036878 0.02209018 0.00519571
  0.01379851 0.05413563 0.02293474 0.01708964 0.01933343 0.02590583
  0.08760197 0.04641684 0.05170624 0.05497634 0.01269901 0.01702163
  0.02645284 0.03841265 0.05547833 0.09143134 0.10673833]]


In [50]:
import tensorflow as tf
# Load the .pb file
converter = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/model6")
# Convert the model
tflite_model = converter.convert()
# Save the .tflite file
with open("/content/converted_model951.tflite", "wb") as f:
   f.write(tflite_model)

In [75]:

import tensorflow as tf
import numpy as np
from PIL import Image


# Charger une image pour la prédiction
image0 = Image.open("/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test/Z_test.jpg")  # Remplacer "image.jpg" par le chemin de votre image

# Prétraiter l'image pour l'adapter aux exigences du modèle
image0 = image0.resize((224, 224))  # Redimensionner l'image selon les besoins du modèle
image_array0 = np.array(image0)  # Convertir l'image en tableau numpy
image_array0 = tf.keras.applications.mobilenet_v3.preprocess_input(image_array0)  # Prétraiter l'image

# Effectuer la prédiction
predictions = model.predict(np.expand_dims(image_array0, axis=0))

# Afficher les résultats
print(predictions)

# DONE.


1/1 [==============================] - 0s 98ms/step
[[5.1824700e-06 1.0633407e-06 1.2188146e-06 1.0741206e-05 9.5482792e-06
  1.0360882e-07 5.7243828e-06 4.7274378e-05 1.0434390e-03 9.8209122e-05
  1.6142345e-04 1.2965791e-04 2.4653590e-07 2.2006832e-06 3.2813437e-05
  5.0430353e-05 1.4185560e-05 3.2344093e-03 6.5756490e-06 4.8163861e-07
  8.5360149e-04 1.5212652e-04 1.5104294e-07 1.3349741e-04 1.3051673e-05
  9.9398535e-01 2.8787792e-06 8.1447877e-07 3.6374768e-06]]


In [76]:
# Sélectionner l'index de la classe avec la probabilité la plus élevée
index_classe_max = predictions.argmax()

# Obtenir la probabilité de la classe prédite en pourcentage
probabilite_max = predictions.max() * 100

# Afficher le résultat
print("Classe prédite:", index_classe_max)
print("Probabilité de la classe prédite en pourcentage:", probabilite_max)

Classe prédite: 25
Probabilité de la classe prédite en pourcentage: 99.39853549003601


In [30]:
import tensorflow as tf

# Charger le modèle TensorFlow sauvegardé
chemin_modele_sauvegarde = "/content/SignLanguageALSClassifier16.h5"
modele = tf.saved_model.load(chemin_modele_sauvegarde)

# Convertir le modèle TensorFlow en TFLite avec quantification et normalisation
convertisseur = tf.lite.TFLiteConverter.from_saved_model(chemin_modele_sauvegarde)
convertisseur.optimizations = [tf.lite.Optimize.DEFAULT]  # Appliquer la quantification et la normalisation
tflite_modele = convertisseur.convert()

# Enregistrer le modèle TFLite
chemin_tflite_modele = "/content/modele6.tflite"
with open(chemin_tflite_modele, 'wb') as f:
    f.write(tflite_modele)


OSError: SavedModel file does not exist at: /content/SignLanguageALSClassifier16.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
results = model.evaluate(test_images)

In [ ]:
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
# DRAW Results
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [19]:
import tensorflow as tf

# Chargement du modèle .pb
model = tf.keras.models.load_model('/content/SignLanguageALSClassifier16.h5')

# Conversion en TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Écriture du modèle TFLite dans un fichier
with open('modele5.tflite', 'wb') as f:
    f.write(tflite_model)

print("Le modèle a été compressé avec succès en format TFLite.")


Le modèle a été compressé avec succès en format TFLite.


In [29]:
import numpy as np
import tensorflow as tf

# Charger le modèle TFLite
interpreter = tf.lite.Interpreter(model_path="/content/modele5.tflite")
interpreter.allocate_tensors()

# Charger une image pour la prédiction
image = Image.open("/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test/Z_test.jpg")  # Remplacer "image.jpg" par le chemin de votre image

# Prétraiter l'image pour l'adapter aux exigences du modèle
image = image.resize((224, 224))  # Redimensionner l'image selon les besoins du modèle
image_array = np.array(image)  # Convertir l'image en tableau numpy
image_array = tf.keras.applications.mobilenet_v3.preprocess_input(image_array)  # Prétraiter l'image

# Entrer les données d'entrée dans le modèle TFLite
input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], np.expand_dims(image_array, axis=0))

# Effectuer l'inférence
interpreter.invoke()

# Obtenir les résultats de l'inférence
output_details = interpreter.get_output_details()
output = interpreter.get_tensor(output_details[0]['index'])

# Sélectionner l'index de la classe avec la probabilité la plus élevée
index_classe_max = np.argmax(output)

# Obtenir la probabilité de la classe prédite en pourcentage
probabilite_max = np.max(output) * 100

# Afficher le résultat
print("Classe prédite:", index_classe_max)
print("Probabilité de la classe prédite en pourcentage:", probabilite_max)

# DONE.


ValueError: Cannot set tensor: Got value of type UINT8 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 

In [28]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Charger le modèle TFLite
interpreter = tf.lite.Interpreter(model_path="/content/modele5.tflite")
interpreter.allocate_tensors()

# Obtenir les détails des données d'entrée et de sortie
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Charger une image pour la prédiction
image = Image.open("/content/drive/MyDrive/ASL/asl_alphabet_test/asl_alphabet_test/B_test.jpg")  # Remplacer "image.jpg" par le chemin de votre image

# Prétraiter l'image pour l'adapter aux exigences du modèle
image = image.resize((224, 224))  # Redimensionner l'image selon les besoins du modèle
image_array = np.array(image)  # Convertir l'image en tableau numpy
image_array = (image_array.astype(np.float32) / 127.5) - 1  # Normaliser l'image

# Vérifier le type de données d'entrée attendu par le modèle
if input_details[0]['dtype'] == np.float32:
    image_array = np.float32(image_array)

# Entrer les données d'entrée dans le modèle TFLite
interpreter.set_tensor(input_details[0]['index'], np.expand_dims(image_array, axis=0))

# Effectuer l'inférence
interpreter.invoke()

# Obtenir les résultats de l'inférence
output_data = interpreter.get_tensor(output_details[0]['index'])

# Sélectionner l'index de la classe avec la probabilité la plus élevée
index_classe_max = np.argmax(output_data)

# Obtenir la probabilité de la classe prédite en pourcentage
probabilite_max = np.max(output_data) * 100

# Afficher le résultat
print("Classe prédite:", index_classe_max)
print("Probabilité de la classe prédite en pourcentage:", output)


Classe prédite: 28
Probabilité de la classe prédite en pourcentage: [[0.03024656 0.00826467 0.00588355 0.04896422 0.02545064 0.01389448
  0.01670773 0.04814664 0.01265362 0.02036878 0.02209018 0.00519571
  0.01379851 0.05413563 0.02293474 0.01708964 0.01933343 0.02590583
  0.08760197 0.04641684 0.05170624 0.05497634 0.01269901 0.01702163
  0.02645284 0.03841265 0.05547833 0.09143134 0.10673833]]
